# Import

In [ ]:
import sys
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy.ndimage.interpolation  as interplt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image

ROOT_DIR = os.path.abspath("../src")
sys.path.append(ROOT_DIR)

import utils
import constants

# Init

In [ ]:
imgs0=[]
imgs1=[]
imgs2=[]
uvd_gt=[]
uvd_centre=[]
uvd_norm_gt=[]
new_file_names=[]
xyz_gt=[]

# Load Data

In [ ]:
train_pairs, test_pairs = utils.get_data_list('depth')

In [ ]:
file_name = [i for i,j in train_pairs]
xyz_gt = [j for i,j in train_pairs]
xyz_gt = np.reshape(xyz_gt,(-1, 21, 3))
uvd_gt = utils.xyz2uvd_batch_depth(xyz_gt)
print(xyz_gt.shape)
# new_xyz_gt = utils.uvd2xyz_batch_depth(uvd_gt)
# print(np.allclose(uvd_gt[0], utils.xyz2uvd_depth(np.reshape(xyz_gt[0], (21, 3))))) 
# print(np.allclose(xyz_gt, new_xyz_gt))

In [ ]:
# for i in range(xyz_gt.shape[0]):
i = 0

depth = Image.open(file_name[i])
depth = np.asarray(depth, dtype='uint16')
fig, ax = plt.subplots()
ax.imshow(depth)
utils.visualize_joints_2d(ax, uvd_gt[i][constants.REORDER], joint_idxs=False)

# Process - Get all points within given bounds

In [ ]:
# collect the points within the axis-aligned BBOX of the hand
xyz_hand_gt = xyz_gt[i].copy()

xyz_hand_gt_bounds = np.array([np.min(xyz_hand_gt[:, 0]), np.max(xyz_hand_gt[:, 0]),
                           np.min(xyz_hand_gt[:, 1]), np.max(xyz_hand_gt[:, 1]),
                           np.min(xyz_hand_gt[:, 2]), np.max(xyz_hand_gt[:, 2])])
print('xyz_hand_gt_bounds=', xyz_hand_gt_bounds)

uvd = utils.depth_to_uvd(depth)
uvd = uvd.reshape(uvd.shape[0]*uvd.shape[1], 3)
all_points = utils.uvd2xyz_depth(uvd)
print('all_points.shape=',all_points.shape)

all_points[:, 2] = all_points[:, 2] #m to mm

all_points_bounds = np.array([np.min(all_points[:, 0]), np.max(all_points[:, 0]),
                           np.min(all_points[:, 1]), np.max(all_points[:, 1]),
                           np.min(all_points[:, 2]), np.max(all_points[:, 2])])

print('all_points_bounds=', all_points_bounds)

bound_offset = 20 # how much looser we want to make the bbox
mask = np.array([1, 0, 1, 0, 1, 0], dtype=bool)
xyz_hand_gt_bounds[mask] -= bound_offset
xyz_hand_gt_bounds[~mask] += bound_offset
print('loose xyz_hand_gt_bounds=', xyz_hand_gt_bounds)

mask_x = (all_points[:, 0] >= xyz_hand_gt_bounds[0]) & (all_points[:, 0] <= xyz_hand_gt_bounds[1]) #x
mask_y = (all_points[:, 1] >= xyz_hand_gt_bounds[2]) & (all_points[:, 1] <= xyz_hand_gt_bounds[3]) #y
mask_z = (all_points[:, 2] >= xyz_hand_gt_bounds[4]) & (all_points[:, 2] <= xyz_hand_gt_bounds[5]) #z
hand_points_xyz = all_points[mask_x & mask_y & mask_z] # points within the bbox
print('hand_points_xyz.shape=', hand_points_xyz.shape)

# Process - bbox and mean

In [ ]:
center_joint_idx = 3
ref_z = 1000
bbsize = 260 #124x124 bbox in uvd
img_size = 96

#create bounding box
bbox_xyz = np.array([(bbsize,bbsize,ref_z)])
print('bbox_xyz=', bbox_xyz)
bbox_uvd = utils.xyz2uvd_depth(bbox_xyz)
print(bbox_uvd)
bbox_uvd[0,0] = np.ceil(bbox_uvd[0,0] - constants.X0_DEPTH)
bbox_uvd[0,1] = np.ceil(bbox_uvd[0,1] - constants.Y0_DEPTH)
print('bbox_uvd=', bbox_uvd)

#get mean values
mean_z = xyz_hand_gt[center_joint_idx, 2]
#modify the mean
#offset mean and bring to reference
hand_points_xyz[:,2] += ref_z - mean_z 
xyz_hand_gt[:,2] += ref_z - mean_z 
uvd_hand_gt = utils.xyz2uvd_depth(xyz_hand_gt)
mean_u = uvd_hand_gt[center_joint_idx, 0]
mean_v = uvd_hand_gt[center_joint_idx, 1]
print('mean_u=', mean_u)
print('mean_v=', mean_v)
print('mean_z=', mean_z)
print('new_mean_z=', xyz_hand_gt[center_joint_idx, 2])

# Process - Get cropped hand

In [ ]:
hand_points_uvd = utils.xyz2uvd_depth(hand_points_xyz)
print('hand_points_uvd.shape', hand_points_uvd.shape)

# Place the hand within the bbox

# U
#offset mean and bring to bbox
hand_points_uvd[:,0] = hand_points_uvd[:,0] - mean_u + bbox_uvd[0,0]/2 
#all points more than bbox now within the bbox
hand_points_uvd[np.where(hand_points_uvd[:,0]>=bbox_uvd[0,0]),0]=bbox_uvd[0,0]-1
#remove negative points
hand_points_uvd[np.where(hand_points_uvd[:,0]<0),0]=0 
save_uvd_hand_gt = uvd_hand_gt.copy()
#offset mean and bring to bbox
uvd_hand_gt[:,0] = (uvd_hand_gt[:,0] - mean_u + bbox_uvd[0,0]/2 ) / bbox_uvd[0,0] 
#remove values >1 and <0
# uvd_hand_gt[np.where(uvd_hand_gt[:,0]>1),0]=1
# uvd_hand_gt[np.where(uvd_hand_gt[:,0]<0),0]=0

# V, similarly
hand_points_uvd[:,1] = hand_points_uvd[:,1] - mean_v + bbox_uvd[0,1]/2
hand_points_uvd[ np.where(hand_points_uvd[:,1]>=bbox_uvd[0,1]),1]=bbox_uvd[0,1]-1
hand_points_uvd[ np.where(hand_points_uvd[:,1]<0),1]=0

uvd_hand_gt[:,1] =( uvd_hand_gt[:,1] - mean_v+bbox_uvd[0,1]/2 ) / bbox_uvd[0,1]
# uvd_hand_gt[ np.where(uvd_hand_gt[:,1]>1),1]=1
# uvd_hand_gt[ np.where(uvd_hand_gt[:,1]<0),1]=0

# Z
hand_points_uvd[:,2] = (hand_points_uvd[:,2] - ref_z + bbsize/2)/bbsize
uvd_hand_gt[:,2] = (uvd_hand_gt[:,2] - ref_z + bbsize/2)/bbsize

#get new hand
new_hand = np.ones((int(bbox_uvd[0,1]),int(bbox_uvd[0,0])),dtype='float32')
new_hand[np.asarray(np.floor(hand_points_uvd[:,1]),dtype='int16'),
         np.asarray(np.floor(hand_points_uvd[:,0]),dtype='int16')] = hand_points_uvd[:,2]
print('new_hand.shape=', new_hand.shape)

r0_tmp = interplt.zoom(new_hand, img_size/bbox_uvd[0,0],order=1, mode='nearest',prefilter=True)
print(bbox_uvd)
print(new_hand.shape)
fig, ax = plt.subplots(1, 2)
ax[0].imshow(r0_tmp)
ax[0].set_title('resized')
ax[1].imshow(new_hand)
ax[1].set_title('new_hand')

# normuvd2xyz

In [ ]:
new_uvd_hand_gt = uvd_hand_gt.copy()

# new_uvd_hand_gt[:, 0] = new_uvd_hand_gt[:, 0]*bbox_uvd[0, 0] + mean_u - bbox_uvd[0,0]/2
# new_uvd_hand_gt[:, 1] = new_uvd_hand_gt[:, 1]*bbox_uvd[0, 1] + mean_v - bbox_uvd[0,1]/2
# new_uvd_hand_gt[:, 2] = new_uvd_hand_gt[:, 2]*bbsize + ref_z - bbsize/2
# new_xyz_hand_gt = utils.uvd2xyz_depth(new_uvd_hand_gt)
# new_xyz_hand_gt[:,2] = new_xyz_hand_gt[:,2] - ref_z + mean_z 

new_xyz_hand_gt, _ = utils.normuvd2xyzuvd_depth(new_uvd_hand_gt, [mean_u, mean_v, mean_z])

new_uvd_gt = utils.xyz2uvd_depth(new_xyz_hand_gt)
print(np.allclose(new_xyz_hand_gt, xyz_gt[0], atol=1e-5))
print(np.allclose(new_uvd_gt, uvd_gt[0]))